# Multi-layer perceptron


In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV
import pickle
import os

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.


In [2]:
model_name = "mlp"
aml_dir = './data/'
train = pd.read_csv(os.path.join(aml_dir, 'nyc_demand_train.csv'), parse_dates=['timeStamp'])
X = train.drop(['demand', 'timeStamp'], axis=1)

Due to the relatively long training times of neural networks, in this example we use one validation set in place of cross-validation. The validation set will be the final 25% of the training dataset. Training takes about 3 minutes.

In [3]:
len_train = int(len(X) * 0.75)
len_valid = len(X) - len_train
test_fold = [-1]*len_train + [0]*len_valid
ps = PredefinedSplit(test_fold)

In [4]:
regr = MLPRegressor(solver='lbfgs', verbose=True)

We first specify a list of network structures to try. Each tuple in this list specifies the number of hidden layers and the number of units in each. For example, (5,0) specifies a network with one hidden layer containing 5 units while (10,10) creates a network with 2 hidden layers, each with 10 units. We then perform grid search to test the various network structures and regularization parameters. 

In [5]:
hidden_layer_size = [(5,), (10,), (15,), (20,), (25,), (30,), (35,), (40,), (10,10), (20,20), (30,30), (40,40)]

In [6]:
param_grid = {'hidden_layer_sizes': hidden_layer_size,
             'alpha': [0.01, 0.1, 1.0, 10.0]}
regr_cv = GridSearchCV(estimator=regr,
            param_grid=param_grid,
            cv=ps,
            scoring='neg_mean_squared_error',
            verbose=2,
            n_jobs=-1)

In [7]:
regr_pipe = Pipeline([('regr_cv', regr_cv)])
regr_pipe.fit(X, y=train['demand'])

Fitting 1 folds for each of 48 candidates, totalling 48 fits
[CV] alpha=0.01, hidden_layer_sizes=(5,) .............................
[CV] alpha=0.01, hidden_layer_sizes=(10,) ............................
[CV] .............. alpha=0.01, hidden_layer_sizes=(5,), total=   1.0s
[CV] alpha=0.01, hidden_layer_sizes=(15,) ............................
[CV] ............. alpha=0.01, hidden_layer_sizes=(10,), total=  11.6s
[CV] alpha=0.01, hidden_layer_sizes=(20,) ............................
[CV] ............. alpha=0.01, hidden_layer_sizes=(15,), total=  14.4s
[CV] alpha=0.01, hidden_layer_sizes=(25,) ............................
[CV] ............. alpha=0.01, hidden_layer_sizes=(20,), total=  12.2s
[CV] alpha=0.01, hidden_layer_sizes=(30,) ............................
[CV] ............. alpha=0.01, hidden_layer_sizes=(25,), total=  18.9s
[CV] alpha=0.01, hidden_layer_sizes=(35,) ............................
[CV] ............. alpha=0.01, hidden_layer_sizes=(30,), total=  27.2s
[CV] alpha=0.01,

[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  8.1min


[CV] ............. alpha=10.0, hidden_layer_sizes=(15,), total=  10.0s
[CV] alpha=10.0, hidden_layer_sizes=(20,) ............................
[CV] ........... alpha=1.0, hidden_layer_sizes=(40, 40), total=  53.9s
[CV] alpha=10.0, hidden_layer_sizes=(25,) ............................
[CV] ............. alpha=10.0, hidden_layer_sizes=(20,), total=  14.2s
[CV] alpha=10.0, hidden_layer_sizes=(30,) ............................
[CV] ............. alpha=10.0, hidden_layer_sizes=(25,), total=  15.9s
[CV] alpha=10.0, hidden_layer_sizes=(35,) ............................
[CV] ............. alpha=10.0, hidden_layer_sizes=(30,), total=  18.8s
[CV] alpha=10.0, hidden_layer_sizes=(40,) ............................
[CV] ............. alpha=10.0, hidden_layer_sizes=(35,), total=  18.1s
[CV] alpha=10.0, hidden_layer_sizes=(10, 10) .........................
[CV] ............. alpha=10.0, hidden_layer_sizes=(40,), total=  25.6s
[CV] alpha=10.0, hidden_layer_sizes=(20, 20) .........................
[CV] .

[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed: 10.5min finished


Pipeline(memory=None,
     steps=[('regr_cv', GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
       error_score='raise',
       estimator=MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1..._jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=2))])

In [8]:
with open(os.path.join(aml_dir, model_name + '.pkl'), 'wb') as f:
    pickle.dump(regr_pipe, f)

Cross validation results

In [9]:
cv_results = pd.DataFrame(regr_pipe.named_steps['regr_cv'].cv_results_)
cv_results.sort_values(by='rank_test_score', inplace=True)
cv_results.head()

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('std_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_alpha,param_hidden_layer_sizes,params,rank_test_score,split0_test_score,split0_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
2,14.337078,0.018942,-5911.094651,-6940.419264,0.01,"(15,)","{'alpha': 0.01, 'hidden_layer_sizes': (15,)}",1,-5911.094651,-6940.419264,0.0,0.0,0.0,0.0
33,30.156150,0.033946,-6232.664230,-7248.436089,1,"(20, 20)","{'alpha': 1.0, 'hidden_layer_sizes': (20, 20)}",2,-6232.664230,-7248.436089,0.0,0.0,0.0,0.0
21,37.313997,0.040670,-6792.659147,-7952.301201,0.1,"(20, 20)","{'alpha': 0.1, 'hidden_layer_sizes': (20, 20)}",3,-6792.659147,-7952.301201,0.0,0.0,0.0,0.0
17,20.024080,0.021181,-6887.305729,-8035.634954,0.1,"(30,)","{'alpha': 0.1, 'hidden_layer_sizes': (30,)}",4,-6887.305729,-8035.634954,0.0,0.0,0.0,0.0
25,14.666589,0.048451,-7142.320120,-8379.513645,1,"(10,)","{'alpha': 1.0, 'hidden_layer_sizes': (10,)}",5,-7142.320120,-8379.513645,0.0,0.0,0.0,0.0
